In [6]:
import sys
sys.path.append('/home/marnugue/Documents/crop_scheduling_optimization')
from src.models.py_crop_engine import PYCropEngine
engine = PYCropEngine()

import time
start_time = time.time()
engine.execute(time_limit=30,
               solution_path="./",
               solver='scip')
print("--- Execution takes %s seconds ---" % (time.time() - start_time))

Scheduling ...


Scheduler find feasible solution!
Solution is optimal!


--- Execution takes 24.146020889282227 seconds ---


In [7]:
import pandas as pd
import plotly.express as px
from datetime import datetime, timedelta

pd.options.mode.copy_on_write = True

START_DATE = datetime(2023,11,2)

In [8]:
df = pd.read_csv("./solution.csv", header=[0,1], index_col=0)

df_list = []
df = df.iloc[1:,:]
for col in df.columns:
    if (df[col] == 0).all():
        df.drop(col,axis=1,inplace=True)
        
def get_vegetable(row):
    l = [idx for idx in row.index if row[idx]]
    if l:
        return l[0]
    else:
        return None

for pos in df.columns.get_level_values(level=0):
    df_pos = df[pos]
    df_pos['Vegetable'] = df_pos.apply(lambda row: get_vegetable(row), axis=1)
    df_pos['Event'] = (df_pos != df_pos.shift(1)).T.any()
    df_pos['Event'] = df_pos['Event'].cumsum()
    df_start = df_pos.reset_index()[['Time','Event']].groupby('Event').first()
    df_end = df_pos.reset_index()[['Time','Event']].groupby('Event').last()
    df_start_end = pd.merge(df_start,
                        df_end,
                        left_index=True,
                        right_index=True,
                        how='inner',
                        suffixes=('_start','_end'))
    df_start_end['Time_end'] = df_start_end['Time_end'] + 1
    df_start_end = df_start_end.apply(lambda row: START_DATE + pd.to_timedelta(row,'d'))
    df_pos = pd.merge(df_pos,
                    df_start_end,
                    left_on='Event',
                    right_index=True,
                    how='inner')
    df_pos.drop_duplicates(inplace=True)
    df_pos['Pos'] = pos
    df_list.append(df_pos[['Pos','Vegetable','Time_start','Time_end']])
df = pd.concat(df_list,ignore_index=True)

In [9]:
df

Vegetable,Pos,Vegetable,Time_start,Time_end
0,P0,H1,2023-11-03,2023-11-10
1,P0,H2,2023-11-10,2023-11-17
2,P0,H1,2023-11-17,2023-11-22
3,P0,H3,2023-11-22,2023-11-26
4,P0,H2,2023-11-26,2023-11-29
...,...,...,...,...
88,P3,H3,2023-11-18,2023-11-20
89,P3,H2,2023-11-20,2023-11-23
90,P3,H3,2023-11-23,2023-11-25
91,P3,H2,2023-11-25,2023-11-28


In [10]:
fig = px.timeline(df, x_start='Time_start',
                  x_end='Time_end',
                  y='Pos',
                  color='Vegetable')
fig.show()